In [ ]:
import os
import sys
import scipy as sp
import numpy as np
import pandas as pd
import xarray as xr
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib
import statsmodels.api as sm
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
matplotlib.rc_file('../rc_file')
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
sys.path.append("..")
from tqdm import tqdm, tqdm_notebook
from maps import rect_polygon
from paths import path_data, path_results, path_prace, file_HadISST
from paths import file_ex_ocn_ctrl, file_ex_ocn_lpd
from paths import file_RMASK_ocn, file_RMASK_ocn_rect
from filters import lowpass
from regions import AMO_mask, boolean_mask
from timeseries import IterateOutputCESM
from xr_DataArrays import xr_AREA
from xr_regression import xr_quadtrend, xr_lintrend
from SST_index_generation import times_ctrl, times_lpd, times_had
from bb_analysis_timeseries import AnalyzeTimeSeries as ATS
from matplotlib.ticker import NullFormatter
from bd_analysis_indices import AnalyzeIndex as AI

## random time series for illustration

In [ ]:
N=70
t = np.arange(N)
a = np.random.rand(N)
plt.figure(figsize=(2,.8), constrained_layout=True)
plt.plot(t,a)
plt.axis('off')
plt.savefig(f'{path_results}/BBOS19/white_noise', transparent=True)
plt.figure(figsize=(2,.8), constrained_layout=True)
plt.plot(t,lowpass(xr.DataArray(dims='time', coords={'time':t}, data=a),10))
plt.axis('off')
plt.savefig(f'{path_results}/BBOS19/red_noise', transparent=True)

## Detrending with the CMIP ensemble 

In [ ]:
def read_cmip5_ens_txt(fn, n_ens, last_year):
    if last_year==2018: hn = 11
    elif last_year==2005: hn =12
    aa = pd.read_csv(fn, header=hn, delim_whitespace=True, skip_blank_lines=True).rename(columns={'#':'year'}).set_index('year').drop(['member', '0'], axis=1)
#     print(aa)
    df = pd.DataFrame(index=np.arange(1870,last_year+1))
    dt = last_year+1-1870
    for i in range(n_ens):
        df[i] = aa['ensemble'][i*(dt+1):i*(dt+1)+dt].values.astype(np.float64)
    return df

cmip5_nat_ens_fn = f'{path_data}/CMIP5/iglobal_tas_Amon_ens_historicalNat_+++_1870 2005_mean1_anom_a.txt'
cmip5_hist_ens_fn = f'{path_data}/CMIP5/iglobal_tas_Amon_ens_rcp85_+++_1870 2018_mean1_anom_a.txt'

cmip5_nat_ens  = read_cmip5_ens_txt(cmip5_nat_ens_fn , n_ens=67, last_year=2005)
cmip5_hist_ens = read_cmip5_ens_txt(cmip5_hist_ens_fn, n_ens=81, last_year=2018)

In [ ]:
# GMST
hadcrut = xr.open_dataarray(f'{path_data}/HadCRUT/ihad4_krig_v2_0-360E_-90-90N_n_mean1_anom_30.nc', decode_times=False)
gmst_had  = xr.open_dataarray(f'{path_prace}/GMST/GMST_dt_yrly_had.nc', decode_times=False)
gmst_had  = gmst_had.isel({'time':slice(9,158)})
gmst_ctrl = xr.open_dataset(f'{path_prace}/GMST/GMST_ctrl.nc').GMST.isel({'time':slice(50,300)})
gmst_ctrl['time'] = (gmst_ctrl.time/365).astype(dtype=int)
gmst_ctrl_dt = gmst_ctrl - xr_quadtrend(gmst_ctrl)
gmst_lpd  = xr.open_dataset(f'{path_prace}/GMST/GMST_lpd.nc').GMST.isel({'time':slice(0,250)})
gmst_lpd['time'] = (gmst_lpd.time/365).astype(dtype=int)
gmst_lpd_dt = gmst_lpd - xr_quadtrend(gmst_lpd)

cmip5_natural = xr.open_dataarray(f'{path_prace}/GMST/CMIP5_natural.nc', decode_times=False)
cmip5_anthro  = xr.open_dataarray(f'{path_prace}/GMST/CMIP5_anthro.nc' , decode_times=False)
cmip5_all     = xr.open_dataarray(f'{path_prace}/GMST/CMIP5_all.nc'    , decode_times=False)

In [ ]:
cc='g'
for i in range(7):
    plt.figure(figsize=(6,6))
    ax = plt.gca()
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    for label in ax.get_xticklabels():   label.set_color(cc)
    for label in ax.get_yticklabels():   label.set_color(cc)
    ax.spines['left'].set_color(cc)
    ax.set_ylim(-.49,3.2)
    ax.spines['left'].set_bounds(-.3, 3.2)
    ax.spines['bottom'].set_bounds(1870, 2020)
    ax.spines['bottom'].set_color(cc)
    ax.tick_params(labelsize=14, color=cc)
    plt.plot([1870,2020],[10,10], c='k')

    if i>0 and i<4:  plt.plot(hadcrut.time[20:]+1850, hadcrut[20:]+2.4, c='C0')

    if i>1 and i<6:  plt.plot(cmip5_hist_ens.index, cmip5_hist_ens-cmip5_hist_ens.loc[1950:1980].mean(axis=0)+1.6, c='orange', lw=.3, alpha=.15)
    if i>2:  plt.plot(np.arange(1870,2019), cmip5_all    +1.6, c='orange')

    if i>3 and i<6:  plt.plot(cmip5_nat_ens.index, cmip5_nat_ens-cmip5_nat_ens.loc[1950:1980].mean(axis=0)+.8    , c='g'     , lw=.3, alpha=.15)
    if i>4: plt.plot(np.arange(1870,2019), cmip5_natural+ .8, c='g')

    if i>5: 
        ax.text(1945, 1.1, '-', color=cc, fontsize=14, ha='center')
        ax.text(1945,  .3, '=' , color=cc, fontsize=14, ha='center')
        plt.plot(np.arange(1870,2019), cmip5_anthro    , c='r')
    plt.xticks(np.arange(1870,2030,30))
    plt.xlabel('time [years C.E.]'      , fontsize=14, color=cc)
    plt.ylabel('temperature anomaly [K]', fontsize=14, color=cc)
    plt.tight_layout()
#     plt.savefig(f'{path_results}/BBOS19/detrending/detr_{i}', dpi=150, transparent=True)

## AMO signal

In [ ]:
AREA_low = xr_AREA(domain='ocn_low')
AREA_rect = xr_AREA(domain='ocn_rect')
AREA_had = xr_AREA(domain='ocn_had')
Atl_MASK_ctrl = boolean_mask(domain='ocn_rect', mask_nr=6).sel(t_lat=slice(0,60))
Atl_MASK_had = boolean_mask(domain='ocn_had', mask_nr=6).sel(latitude=slice(60,0))

In [ ]:
# raw AMO signal
had_raw = xr.open_dataarray(f'{path_prace}/SST/SST_yrly_had.nc', decode_times=False)
AMO_yrly_raw_had  = (had_raw*AREA_had).where(Atl_MASK_had).sum(dim=['latitude', 'longitude'])/AREA_had.where(Atl_MASK_had).sum(dim=['latitude', 'longitude'])
ctrl_raw = xr.open_dataarray(f'{path_prace}/SST/SST_yrly_rect_ctrl.nc', decode_times=False)[50:,:,:]
AMO_yrly_raw_ctrl = (ctrl_raw*AREA_rect).where(Atl_MASK_ctrl).sum(dim=['t_lat', 't_lon'])/AREA_rect.where(Atl_MASK_ctrl).sum(dim=['t_lat', 't_lon'])

In [ ]:
# %%time
# AMO_monthly_had  = (had*AREA_had).where(Atl_MASK_had).sum(dim=['latitude', 'longitude'])/AREA_had.where(Atl_MASK_had).sum(dim=['latitude', 'longitude'])
# AMO_monthly_ctrl = (ctrl*AREA_rect).where(Atl_MASK_ctrl).sum(dim=['t_lat', 't_lon'])/AREA_rect.where(Atl_MASK_ctrl).sum(dim=['t_lat', 't_lon'])
# AMO_monthly_lpd  = (lpd*AREA_low).where(AMO_mask(domain='ocn_low')).sum(dim=['nlon','nlat'])/AREA_low.where(AMO_mask(domain='ocn_low')).sum(dim=['nlon','nlat'])
# AMO_monthly_had .to_netcdf(f'{path_prace}/SST/AMO_ds_dt_monthly_had.nc')
# AMO_monthly_ctrl.to_netcdf(f'{path_prace}/SST/AMO_ds_dt_monthly_ctrl_51_301.nc')
# AMO_monthly_lpd .to_netcdf(f'{path_prace}/SST/AMO_ds_dt_monthly_lpd_154_404.nc')
AMO_monthly_had  = xr.open_dataarray(f'{path_prace}/SST/AMO_ds_dt_monthly_had.nc', decode_times=False)
AMO_monthly_ctrl = xr.open_dataarray(f'{path_prace}/SST/AMO_ds_dt_monthly_ctrl_51_301.nc', decode_times=False)
AMO_monthly_lpd  = xr.open_dataarray(f'{path_prace}/SST/AMO_ds_dt_monthly_lpd_154_404.nc', decode_times=False)

In [ ]:
cc='w'
for i in range(4):
    plt.figure(figsize=(5.5,2.5))
    ax = plt.gca()
    ax.set_position([1/5.5,.22,4.5/5.5-.01,.77])
    ax.tick_params(labelsize=14, color=cc)
    if i<2:
        ax.set_ylim(21.4,22.7)
        ax.set_yticks([21.5,22,22.5])
        ax.plot(AMO_yrly_raw_had.time/365+1870, AMO_yrly_raw_had)
        ax.spines['left'].set_bounds(21.5, 22.5)
    if i==1:
        ax.plot(np.arange(1870,2019), cmip5_natural+ 22.5, c='g')
        ax.plot(np.arange(1870,2019), cmip5_anthro +21.8, c='r')
    if i==2:
        ax.axhline(0, c='lightgrey', lw=.5)
        ax.set_ylim(-.33,.4)
        ax.set_yticks([-.3,0, .3])
        ax.spines['left'].set_bounds(-.3,.3)
    if i==2:
        ax.plot(np.arange(1870,2019), AMO_monthly_had.groupby_bins(group='time', bins=np.arange(0,150*365,365)).mean())
    if i==3:
        ax.plot(np.arange(1870,2019), AMO_monthly_had.groupby_bins(group='time', bins=np.arange(0,150*365,365)).mean(), lw=.5)
        y = lowpass(AMO_monthly_had.groupby_bins(group='time', bins=np.arange(0,150*365,365)).mean(), 13)
        ax.plot(np.arange(1870,2019), y, c='C0')
        ax.fill_between(np.arange(1870,2019), 0, y, where=y>0, alpha=.3)
        ax.fill_between(np.arange(1870,2019), 0, y, where=y<0, alpha=.3, color='C3')
        
    for label in ax.get_xticklabels():   label.set_color(cc)
    for label in ax.get_yticklabels():   label.set_color(cc)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_bounds(1870, 2020)
    ax.spines['left'].set_color(cc)
    ax.spines['bottom'].set_color(cc)
    ax.set_xlim(1865, 2030)
    ax.set_xticks(np.arange(1870,2030,30))
    ax.set_ylabel('AMO index [K]', fontsize=14, color=cc)
    ax.set_xlabel('time [years C.E.]', fontsize=14, color=cc)
    plt.savefig(f'{path_results}/BBOS19/detrending/AMO_had_{i}', transparent=True)


    plt.figure(figsize=(8.5,2.5))
    ax = plt.gca()
    ax.set_position([1/8.5,.22,7.5/8.5-.01,.77])
    ax.tick_params(labelsize=14, color=cc)
    if i<2:  
        ax.set_ylim(21.9,23.1)
        ax.set_yticks([22,22.5, 23])
        plt.plot(AMO_yrly_raw_ctrl.time, AMO_yrly_raw_ctrl)
        ax.spines['left'].set_bounds(22, 23)
    if i==1:
        ax.plot(AMO_yrly_raw_ctrl.time, xr_quadtrend(AMO_yrly_raw_ctrl), c='yellow')
    if i>1:
        ax.axhline(0, c='lightgrey', lw=.5)
        ax.set_ylim(-.3,.35)
        ax.set_yticks([-.25,0, .25])
        ax.spines['left'].set_bounds(-.25,.25)
    if i==2:
        ax.plot(AMO_yrly_raw_ctrl.time, AMO_yrly_raw_ctrl - xr_quadtrend(AMO_yrly_raw_ctrl))
    if i==3:
        ax.plot(AMO_yrly_raw_ctrl.time, AMO_yrly_raw_ctrl - xr_quadtrend(AMO_yrly_raw_ctrl), lw=.5)
        y = lowpass(AMO_yrly_raw_ctrl - xr_quadtrend(AMO_yrly_raw_ctrl), 13)
        ax.plot(AMO_yrly_raw_ctrl.time, y, c='C0')
        ax.fill_between(AMO_yrly_raw_ctrl.time, 0, y, where=y>0, alpha=.3)
        ax.fill_between(AMO_yrly_raw_ctrl.time, 0, y, where=y<0, alpha=.3, color='C3')
    for label in ax.get_xticklabels():   label.set_color(cc)
    for label in ax.get_yticklabels():   label.set_color(cc)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_bounds(50, 300)
    ax.spines['left'].set_color(cc)
    ax.spines['bottom'].set_color(cc)
    ax.set_xlim(45,305)
    ax.set_ylabel('AMO index [K]', fontsize=14, color=cc)
    ax.set_xlabel('time [model years]', fontsize=14, color=cc)
    plt.savefig(f'{path_results}/BBOS19/detrending/AMO_ctrl_{i}', transparent=True)


In [ ]:
cc='w'
for i in range(3):
    plt.figure(figsize=(5.5,2.5))
    ax = plt.gca()
    ax.set_position([1/5.5,.22,4.5/5.5-.01,.77])
    ax.tick_params(labelsize=14, color=cc)
    if i<1:
        ax.set_ylim(21.4,22.7)
        ax.set_yticks([21.5,22,22.5])
        ax.plot(AMO_yrly_raw_had.time/365+1870, AMO_yrly_raw_had)
        ax.plot(AMO_yrly_raw_had.time/365+1870, xr_lintrend(AMO_yrly_raw_had))
        ax.spines['left'].set_bounds(21.5, 22.5)
    if i>0:
        ax.axhline(0, c='lightgrey', lw=.5)
        ax.set_ylim(-.33,.4)
        ax.set_yticks([-.3,0, .3])
        if i==1:
            ax.plot(np.arange(1870,2019), AMO_yrly_raw_had-xr_lintrend(AMO_yrly_raw_had))
        if i==2:
            
            ax.plot(np.arange(1870,2019), AMO_yrly_raw_had-xr_lintrend(AMO_yrly_raw_had), lw=.5)
            y = lowpass(AMO_yrly_raw_had-xr_lintrend(AMO_yrly_raw_had), 13)
            ax.plot(np.arange(1870,2019), y, c='C0')
            ax.fill_between(np.arange(1870,2019), 0, y, where=y>0, alpha=.4)
            ax.fill_between(np.arange(1870,2019), 0, y, where=y<0, color='C3', alpha=.4)
        ax.spines['left'].set_bounds(-.3,.3)
    for label in ax.get_xticklabels():   label.set_color(cc)
    for label in ax.get_yticklabels():   label.set_color(cc)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_bounds(1870, 2020)
    ax.spines['left'].set_color(cc)
    ax.spines['bottom'].set_color(cc)
    ax.set_xlim(1865, 2030)
    ax.set_xticks(np.arange(1870,2030,30))
    ax.set_ylabel('AMO index [K]', fontsize=14, color=cc)
    ax.set_xlabel('time [years C.E.]', fontsize=14, color=cc)
    plt.savefig(f'{path_results}/BBOS19/detrending/AMO_had_lin_{i}', transparent=True)

In [ ]:
had  = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_ds_dt_had.nc', decode_times=False)
ctrl = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_ds_dt_ctrl_51_301.nc', decode_times=False)
lpd  = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_ds_dt_lpd_154_404.nc', decode_times=False)

In [ ]:
cc='g'
f, ax = plt.subplots(1, 3, figsize=(12,2.3), sharey=True, constrained_layout=True, gridspec_kw={'width_ratios':[1.5,2.5,2.5]})
for i in range(3):
    ax[i].spines['left'].set_color(cc)
    ax[i].spines['bottom'].set_color(cc)
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['left'].set_bounds(-.2,.2)
    ax[i].spines['top'].set_visible(False)
    ax[i].set_ylim(-.22,.22)
    ax[i].axhline(0, c='lightgrey', lw=.5)
    ax[i].tick_params(labelsize=14, color=cc)
    ax[i].set_xticks(np.arange([1900,50,150][i],[2020,301,401][i], 50))
    ax[i].set_xticklabels(np.arange([1900,50,150][i],[2020,301,401][i], 50), color=cc)
    ax[i].spines['bottom'].set_bounds([1870,50,150][i],[2020, 300, 400][i])
ax[0].plot(AMO_monthly_had.time/365+1870, lowpass(AMO_monthly_had , 12*13))
ax[1].plot(AMO_monthly_ctrl.time        , lowpass(AMO_monthly_ctrl, 12*13))
ax[2].plot(AMO_monthly_lpd.time/365     , lowpass(AMO_monthly_lpd , 12*13))
ax[1].set_xlabel('time [years]', fontsize=14, c=cc)
ax[0].set_ylabel('AMO index [K]', fontsize=14, c=cc)
ax[0].set_yticks(np.arange(-.2,.3,.1))
ax[0].set_yticklabels(np.arange(-2,3,1)/10, color=cc)
# plt.savefig(f'{path_results}/BBOS19/AMO_time_series', transparent=True)

In [ ]:
%%time
AMO_spectra = {}
for i, run in enumerate(['had', 'ctrl', 'lpd']):
    ts = [AMO_monthly_had, AMO_monthly_ctrl, AMO_monthly_lpd][i]
    AMO_spectra[f'spec_{run}'] = ATS(lowpass(ts,13*12)[7*12:-7*12]).spectrum()
    AMO_spectra[f'redn_{run}'] = ATS(ts).mc_ar1_spectrum(filter_type='lowpass', filter_cutoff=13*12)

In [ ]:
cc='w'
f, ax = plt.subplots(1, 3, figsize=(12,2.3), sharey=True)
for i, run in enumerate(['had', 'ctrl', 'lpd']):
    ax[i].set_position([.06+i*.31,.25,.3,.73])
    ax[i].spines['left'].set_color(cc)
    ax[i].spines['bottom'].set_color(cc)
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['left'].set_bounds(5e-2,5)
    ax[i].spines['top'].set_visible(False)
    ax[i].tick_params(labelsize=14, color=cc)
    ax[i].set_xscale('log', basex=2)
    ax[i].set_yscale('log', basey=2)
    
    (spec, freq, jackknife) = AMO_spectra[f'spec_{run}']
    AR_spectrum = AMO_spectra[f'redn_{run}']
    ax[i].fill_between(AR_spectrum[1,:]*12, AR_spectrum[2,:], AR_spectrum[3,:], fc=f'C3', alpha=.3, ec=None)
    ax[i].plot(freq*12, spec)
    ax[i].set_xlim(1/150,1/13)
    ax[i].set_ylim(bottom=4e-2, top=5.2)
    ax[i].set_xticks([1/2**n for n in np.arange(4, 8)])
    ax[i].set_yticks([2**float(n) for n in np.arange(-4,3,2)])
    ax[i].set_xticklabels([f'1/{2**n}' for n in np.arange(4, 8)])
    ax[i].set_yticklabels([r'$\dfrac{1}{16}$', r'$\dfrac{1}{4}$', '1', '4'])
    for label in ax[i].get_xticklabels():   label.set_color(cc)
    for label in ax[i].get_yticklabels():   label.set_color(cc)

ax[1].set_xlabel(r'frequency [1/years]', c=cc, fontsize=14)
ax[0].set_ylabel('AMO spectral power    '  , c=cc, fontsize=14)
plt.savefig(f'{path_results}/BBOS19/AMO_spectra', transparent=True)

## AMO regression pattern

In [ ]:
# for i, run in enumerate(tqdm(['had', 'ctrl', 'lpd'])):
#     if i<2:  continue
#     if run=='had' :  ts = ''
#     if run=='ctrl':  ts = '_51_301'
#     if run=='lpd' :  ts = '_154_404'
#     autocorr = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_autocorrelation_{run}{ts}.nc', decode_times=False)
#     SST_dt = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_ds_dt_{run}{ts}.nc', decode_times=False)
#     index = xr.open_dataarray(f'{path_prace}/SST/AMO_ds_dt_monthly_{run}{ts}.nc', decode_times=False)
#     fn_out = f'{path_prace}/SST/AMO_monthly_regr_{run}{ts}.nc'
#     AI().calculate_regression(SST_dt=SST_dt, index=index, tavg='monthly', fn_out=fn_out, autocorr=autocorr)
regr_had  = xr.open_dataset(f'{path_prace}/SST/AMO_monthly_regr_had.nc')
regr_ctrl = xr.open_dataset(f'{path_prace}/SST/AMO_monthly_regr_ctrl_51_301.nc')
regr_lpd  = xr.open_dataset(f'{path_prace}/SST/AMO_monthly_regr_lpd_154_404.nc')

In [ ]:
f = plt.figure(figsize=(15,2.7), constrained_layout=False)
for i, run in enumerate(['had', 'ctrl', 'lpd']):
    regr = [regr_had, regr_ctrl, regr_lpd][i]
    ax = f.add_subplot(1, 4, i+1, projection=ccrs.Robinson(central_longitude=-60))
    ax.set_position([.01+i*.31,.015,.3,.955])
    xa = regr.slope
    if run=='had':
        lats = xa.latitude
        lons = xa.longitude
        lons, lats = np.meshgrid(lons, lats)
    elif run=='ctrl':
        lats = xa.t_lat
        lons = xa.t_lon
        lons, lats = np.meshgrid(lons, lats)
    elif run=='lpd':
        lats = xa.TLAT
        lons = xa.TLONG


    im = ax.pcolormesh(lons, lats, xa.values, cmap='RdBu_r',
                       vmin=-.4, vmax=.4,
                       transform=ccrs.PlateCarree())
    ax.add_feature(cartopy.feature.LAND, zorder=2, edgecolor='black', facecolor='grey')
    
    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False)
    gl.ylocator = matplotlib.ticker.FixedLocator([-90, -60, -30, 0, 30, 60, 90])
    gl.xlocator = matplotlib.ticker.FixedLocator([-180, -120, -60, 0, 60, 120, 180])
    
    if i==2:
        ax = f.add_subplot(1, 4, 4)
        ax.set_position([.94,.1,.02,.8])
        cbar = plt.colorbar(im, cax=ax, fraction=.1, shrink=.8, pad=.05, orientation='vertical', ticks=[-.3, 0, .3])
        cbar.ax.set_yticklabels([-.3,0,.3], fontsize=14, color=cc)

#     f.align_xlabels()
#     plt.tight_layout()
plt.savefig(f'{path_results}/BBOS19/regression_maps', transparent=True)

In [ ]:
regr_had.pval.where(regr_had.pval<.01).plot()

In [ ]:
regr_ctrl.pval.where(regr_ctrl.pval<.01).plot()

In [ ]:
regr_lpd.pval.where(regr_lpd.pval>.99).plot()

In [ ]:
xr.open_dataset(file_HadISST)

## video: globe focused on North Atlantic with AMO time series

In [ ]:
MASK_AMO_had = boolean_mask(domain='ocn_had', mask_nr=6) + boolean_mask(domain='ocn_had', mask_nr=8)
MASK_AMO_had = MASK_AMO_had.where(MASK_AMO_had_anti.latitude<60, 0)
MASK_AMO_had = MASK_AMO_had.where(MASK_AMO_had_anti.latitude>0, 0)
MASK_AMO_had_anti = -MASK_AMO_had+1

In [ ]:
da = xr.open_dataset(file_HadISST, decode_times=False).sst
da = da.groupby_bins(group='time', bins=np.arange(0,150*365,365)).mean(dim='time')

In [ ]:
# 2:15
# `ffmpeg -framerate 12 -start_number 1870 -i SST_AMO_area_%03d.png -r 8 -vcodec png SST_AMO.mov`
for i in tqdm(range(len(AMO_yrly_raw_had.time))):
    fig = plt.figure(figsize=(7,7))

    ax = fig.add_subplot(211, projection=ccrs.NearsidePerspective(central_longitude=-40.0, central_latitude=40.0))
    ax.set_position([1/7,1.9/7,5/7,5/7])

    lats = da.latitude
    lons = da.longitude
    lons, lats = np.meshgrid(lons, lats)

    ax.pcolormesh(lons, lats, da[i,:,:].where(da[i,:,:]>-3), cmap='Spectral_r',
                  vmin=-1, vmax=28, zorder=1,
                  transform=ccrs.PlateCarree())
    ax.pcolormesh(lons, lats, MASK_AMO_had_anti, cmap='Greys',
                  vmin=0, vmax=1, zorder=2, alpha=.15, edgecolors=None,
                  transform=ccrs.PlateCarree())
    ax.add_feature(cartopy.feature.LAND, zorder=3, edgecolor='black', facecolor='grey')

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False)
    gl.ylocator = matplotlib.ticker.FixedLocator([-90, -60, -30, 0, 30, 60, 90])
    gl.xlocator = matplotlib.ticker.FixedLocator([-180, -120, -60, 0, 60, 120, 180])

    ax = fig.add_subplot(221)
    ax.patch.set_alpha(0.0)
    ax.set_position([.12,.05,.86,1.8/7])
    for label in ax.get_xticklabels():   label.set_color('w')
    for label in ax.get_yticklabels():   label.set_color('w')
    ax.spines['left'].set_color('w')
    ax.spines['bottom'].set_color('w')
    ax.tick_params(labelsize=13, color='w')
    plt.plot(AMO_yrly_raw_had.time[:i+1]/365+1870, AMO_yrly_raw_had[:i+1], zorder=0)
    ax.set_ylabel(r'$\langle$SST$\rangle$ [$^\circ$C]', fontsize=16, color='w')
    
    ax.set_xlim((1865,2023))
    ax.set_ylim((21.45,22.7))
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    # plt.tight_layout()
    plt.savefig(f'{path_results}/BBOS19/SST_AMO_video/SST_AMO_area_{int(AMO_yrly_raw_had.time[i]/365+1870)}', transparent=True)
    plt.close()
#     plt.ioff()


## video: comparing SST with different resolution POP to observations

In [ ]:
# NOAA High Resolution SST data provided by the NOAA/OAR/ESRL PSD, Boulder, Colorado, USA
# from their Web site at https://www.esrl.noaa.gov/psd/

NOAA_daily = xr.open_dataarray(f'{path_prace}/NOAA_SST/sst.day.mean.2018.nc')
ctrl_daily = xr.open_mfdataset('/projects/0/prace_imau/prace_2013081679/cesm1_0_4/spinup_pd_maxcores_f05_t12/OUTPUT/ocn/hist/daily/spinup_pd_maxcores_f05_t12.pop.h.nday1.0300*.nc',
                               combine='nested', concat_dim='time').SST
lpd_daily  = xr.open_mfdataset('/projects/0/acc/cesm/cesm1_1_2/spinup_B_2000_cam5_f09_g16/OUTPUT/ocn/hist/daily/spinup_B_2000_cam5_f09_g16.pop.h.nday1.0500-*.nc',
                               combine='nested', concat_dim='time').SST

In [ ]:
t = np.roll(np.linspace(0,1,365), 80)
y = 70*np.sin(2*np.pi*t)+15
y2 = 50*np.sin(2*np.pi*t)-10

x = 360*t-220
x2 = 360*t-150

ax = plt.axes(projection=ccrs.PlateCarree())
ax.stock_img()
# ax = plt.axes(projection=proj)
plt.scatter(x[0], y[0], transform=ccrs.PlateCarree())
plt.scatter(x[180], y[180], transform=ccrs.PlateCarree())
plt.plot(x, y, transform=ccrs.PlateCarree())
plt.plot(x2, y2, transform=ccrs.PlateCarree())

In [ ]:
daterange = pd.date_range('2018-01-01', '2018-12-31')
MASK_ctrl = boolean_mask(domain='ocn', mask_nr=0)
for j in tqdm_notebook(range(len(daterange))):
    fn = f'{path_results}/BBOS19/resolution_video/SST_obs_high_low_{j}.png'
    try:
        assert os.path.exists(fn)
        print(j)
        if j>270: assert 1==0
    except:
        fig = plt.figure(figsize=(15, 5))
        proj = ccrs.NearsidePerspective(central_longitude=x[j], central_latitude=y[j], satellite_height=3578580)
        for i, run in enumerate(['had', 'ctrl', 'lpd']):
            da = [NOAA_daily, ctrl_daily, lpd_daily][i]
            if i==0: lons, lats = np.meshgrid(da.lon, da.lat)
            if i>0:  lats, lons = da.TLAT, da.TLONG

            if i==1:  da_ = da.isel(time=j).where(MASK_ctrl)
            else:     da_ = da.isel(time=j)
            ax = fig.add_subplot(3, 1, i+1, projection=proj)
            ax.add_feature(cartopy.feature.LAND, zorder=2, edgecolor='black', facecolor='grey')
            ax.set_position([.01+i/3,.015,.32,.955])
            im = ax.pcolormesh(lons, lats, da_.values,
                                cmap='Spectral_r', vmin=-1.8, vmax=31,
                                transform=ccrs.PlateCarree(),
                                )
            gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False)
            gl.n_steps = 90
            gl.ylocator = matplotlib.ticker.FixedLocator(np.arange(-90,100,30))
            gl.xlocator = matplotlib.ticker.FixedLocator(np.arange(0,370,30))
        fig.text(.02, .92, daterange[j].strftime('%b %d'), fontsize=14, color='lightgrey')
        fig.text(.02, .88, daterange[j].strftime('%Y')   , fontsize=14, color='lightgrey')
        plt.savefig(fn, transparent=True)
        plt.close()

## volume integrated OHC variability

In [ ]:
ctrl_qd = xr.open_dataset(f'{path_prace}/OHC/OHC_integrals_ctrl_qd.nc', decode_times=False)
lpd_qd  = xr.open_dataset(f'{path_prace}/OHC/OHC_integrals_lpd_qd.nc' , decode_times=False)

In [ ]:
cc='g'
# cc='w'
f, ax = plt.subplots(1,2, figsize=(10,3), sharey=True, constrained_layout=True)
for i, qd in enumerate([ctrl_qd, lpd_qd]):
#     ax[i].set_position([.06+i*.31,.25,.3,.73])
    ax[i].spines['left'].set_color(cc)
    ax[i].spines['bottom'].set_color(cc)
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['left'].set_bounds(-27,27)
    ax[i].spines['bottom'].set_bounds([50,150][i], [300, 400][i])
    ax[i].spines['top'].set_visible(False)
    ax[i].tick_params(labelsize=14, color=cc)
    ax[i].axhline(0, color='lightgrey')
    ax[i].set_xlim([45,149][i], [305, 409][i])
    for label in ax[i].get_xticklabels():   label.set_color(cc)
    for label in ax[i].get_yticklabels():   label.set_color(cc)
    ax[i].plot(qd.time, lowpass(qd.OHC_Global_Ocean  /1e21,13), label='Global')
    ax[i].plot(qd.time, lowpass(qd.OHC_Atlantic_Ocean/1e21,13), label='Atlantic')
    ax[i].set_xlabel('time [model years]', fontsize=14, color=cc)
ax[0].set_ylabel('heat anomaly [ZJ]', fontsize=14, color=cc)
leg = ax[1].legend(ncol=2, fontsize=14, frameon=False, loc=8)
for line, text in zip(leg.get_lines(), leg.get_texts()):  text.set_color(line.get_color())
# plt.savefig(f'{path_results}/BBOS19/OHC_anomalies', transparent=True)

## lead-lag regression

In [ ]:
def merge_time_series(run):
    """creats a pandas dataframe of the GMST, index, and OHC time series"""
    
    # GMST
    if run=='had':
        gmst = xr.open_dataarray(f'{path_prace}/GMST/GMST_dt_yrly_had.nc', decode_times=False)
        gmst.name = 'GMST'
        gmst  = gmst.isel({'time':slice(9,158)})
    else:
        gmst = xr.open_dataset(f'{path_prace}/GMST/GMST_{run}.nc', decode_times=False).GMST
        gmst['time'] = (gmst.time/365).astype(dtype=int)
        if run=='ctrl':   gmst  = gmst.isel({'time':slice(50,300)})
        elif run=='lpd':  gmst  = gmst.isel({'time':slice(0,250)})
        gmst = gmst - xr_quadtrend(gmst)
    
    # indices
    if run=='had'   :  ts = ''
    elif run=='ctrl':  ts = '_51_301'
    elif run=='lpd' :  ts = '_154_404'
    da_temp = xr.open_dataarray(f'{path_prace}/SST/AMO_dt_raw_{run}{ts}.nc')
    da_temp.name = 'AMO'
    da_temp['time'] = (da_temp.time/365).astype(dtype=int)
    if run=='had':  da_temp['time'] = da_temp['time'] + 1870
    
#     da_temp = xr.merge([gmst, lowpass(da_temp,13)])
    da_temp = xr.merge([lowpass(gmst,13), lowpass(da_temp,13)])
    
    return da_temp.isel({'time':slice(7,-7)}).to_dataframe()

time_series_had  = merge_time_series(run='had')
time_series_ctrl = merge_time_series(run='ctrl')
time_series_lpd  = merge_time_series(run='lpd')

In [ ]:
def lagged_correlation_plot(ax, X, Y, kwargs={}):
    Dt = np.arange(-25,26)
    A = []
    for dt in Dt:
        A.append(X.corr(Y.shift(dt)))
        if dt==0:  print(X.corr(Y.shift(dt)))
    return ax.plot(Dt, A, **kwargs)

f, ax = plt.subplots(1, 3, figsize=(12,3), sharey=True, sharex=True,
                     constrained_layout=True)

cc='g'
kwarg = {'label':'AMO'}
for i, run in enumerate(['had','ctrl','lpd']):
    ts = [time_series_had , time_series_ctrl, time_series_lpd ][i]
    lagged_correlation_plot(ax=ax[i], X=ts.GMST, Y=ts['AMO'], kwargs=kwarg)
    ax[i].spines['left'].set_color(cc)
    ax[i].spines['bottom'].set_color(cc)
    ax[i].spines['left'].set_bounds(-1,1)
    ax[i].set_ylim(-1.05,1.05)
    ax[i].spines['bottom'].set_bounds(-25,25)
    for label in ax[i].get_xticklabels():   label.set_color(cc)
    for label in ax[i].get_yticklabels():   label.set_color(cc)
    ax[i].tick_params(labelsize=14, color=cc)
    ax[i].axhline(0, c='lightgrey', lw=.5)
    ax[i].axvline(0, c='lightgrey', lw=.5)
    ax[i].spines['right'].set_visible(False)
    ax[i].spines['top'].set_visible(False)
    ax[i].text(12.5,.9,'AMO leads', fontsize=14, ha='center', color=cc)
    ax[i].text(-12.5,.9,'GMST leads', fontsize=14, ha='center', color=cc)
    ax[i].set_yticks([-1,0,1])
    ax[i].set_xlabel('lag [years]', fontsize=14, color=cc)
ax[0].set_ylabel('correlation', fontsize=14, color=cc)
# plt.savefig(f'{path_results}/BBOS19/AMO_GMST_lead_lag', transparent=True)

In [ ]:
time_series_lpd.plot()

## OHC Hovmoeller

In [ ]:
ctrl_qd['OHC_levels_Global_Ocean']

In [ ]:
oceans = ['Global', 'Atlantic', 'Pacific', 'Southern']
das = [ctrl_qd, lpd_qd]
maxv = .06

fig = plt.figure(figsize=(12,5), constrained_layout=True)
gs0 = matplotlib.gridspec.GridSpec(1, 2, left=.1, right=.98, bottom=.22, top=.98, wspace=.05, hspace=.045)


# if offset==True: x = (da-da.isel(time=slice(0,30)).mean(dim='time')).T/1e21

name = f'Global_Ocean'    
cc='w'
for j, da in enumerate(range(2)):
    da = lowpass(das[j][f'OHC_levels_{name}'],10)
    x = da.T/1e21
#         X, Y = np.meshgrid(da.time, -da.coords[['depth_t', 'z_t'][j]]/[1, 1e2][j])
    X, Y = np.meshgrid(da.time, da.coords[['depth_t', 'z_t'][j]]/[1, 1e2][j])

    gs00 = matplotlib.gridspec.GridSpecFromSubplotSpec(2, 1, subplot_spec=gs0[j], hspace=0)
    ax_top = fig.add_subplot(gs00[0])
    ax_top.set_ylim((-1500,0))
    ax_top.set_xticks([])
    ax_top.set_yticks([-1500, -1000, -500, 0])

    ax_bot = fig.add_subplot(gs00[1])
    ax_bot.set_ylim((-6000,-1500))
    ax_bot.set_yticks([-6000,-4500,-3000])

    for k, ax in enumerate([ax_top, ax_bot]):
        im = ax.pcolormesh(X, -Y, x, vmin=-maxv, vmax=maxv, cmap=cmocean.cm.balance)
        ax.tick_params(labelsize=14, color=cc)
        for label in ax.get_xticklabels():   label.set_color(cc)
        for label in ax.get_yticklabels():   label.set_color(cc)

    if j==0:
        ax_bot.set_ylabel(f'Global Ocean depth [m]', horizontalalignment = 'left', fontsize=14, color=cc)

    if j==1:
        ax_top.set_yticks([])
        ax_bot.set_yticks([])

    if i==0:
        ax_top.text(.5,1.05,['HIGH', 'LOW'][j], transform=ax_top.transAxes)
    if i==len(oceans)-1:
        ax_bot.set_xlabel('time [model years]', fontsize=14)
    else:
        ax_bot.set_xticks([])


cax = fig.add_axes([0.1, 0.11, 0.88, 0.04])
cbar = fig.colorbar(im, cax=cax, orientation='horizontal', extend='both')
cbar.set_label('OHC anomaly [ZJ/m]', color=cc, fontsize=14)
cbar.ax.xaxis.set_tick_params(color=cc)
plt.setp(plt.getp(cbar.ax.axes, 'xticklabels'), color=cc, fontsize=14)
plt.savefig(f'{path_results}/BBOS19/OHC_vertical_Hovmoeller_0-6km_ctrl_lpd_qd', transparent=True)
